In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
from sklearn.decomposition import PCA
import umap

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from statsmodels import robust
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import os
import matplotlib.cm as cm

# run pca and visualize

import scipy

colors = [
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']



Autosaving every 180 seconds


In [5]:
# visualize recomputed templates over time;
def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
        
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2

    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            temp = np.fromfile(
                fin,
                dtype=data_type,
                count=(n_times * n_channels))
            wfs[ctr] = temp.reshape(n_times, n_channels)
    fin.close()
    return wfs

In [4]:
# RETINAL DATASETS
# load raw data and spike train
root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'

# post spike sorting
temps = np.load('/media/cat/4TBSSD/data/retina/2009/templates_0sec.npy')
print ("templates [n_neurons, n_times, n_channels]: ", temps.shape)

spike_train = np.load('/media/cat/4TBSSD/data/retina/2009/spike_train.npy')
print ("spike_train [times, neuron_ids]: ", spike_train.shape)

# raw data
geom = np.loadtxt('/media/cat/4TBSSD/data/retina/2009/geom.txt')
print ("geom: ", geom.shape)

n_chan = 512


templates [n_neurons, n_times, n_channels]:  (2413, 101, 512)
spike_train [times, neuron_ids]:  (94794370, 2)
geom:  (512, 2)


(28814662, 2)
geom:  (384, 2)


In [5]:
# OPTIONAL TAKES UP A LOT OF MEMORY
if False:
    data = np.fromfile('/media/cat/4TBSSD/data/retina/2009/standardized.bin','float32')
    data2D = data.reshape(-1,n_chan)
    print ("Raw data (2D) shape: ", data2D.shape)

    # visualize raw data; Channels x Time
    sampling_rate = 20000.
    t=np.arange(10000)/sampling_rate
    for k in range(n_chan):
        plt.plot(t,data2D[:10000,k]+k*50,c='black')
    plt.show()   


In [11]:
# visualize templates; x-y geometry by n_times
temps = np.load('/media/cat/4TBSSD/data/synthetic/run30_monotonic_3chans/ground_truth/templates_ground_truth.npy')
max_chans = temps.ptp(1).argmax(1)
unit = 0
max_chan = max_chans[unit]
print ("Unit: ", unit, ' , max chan: ', max_chan)
plt.plot((geom[:,0][:,None]+np.arange(101)/2.).T, 
         temps[unit] +
         geom[:,1][None]/3.,c='black')
plt.text(geom[max_chan,0],geom[max_chan,1]/3.,str(max_chan),fontsize=10)

plt.show()

Unit:  0  , max chan:  251


In [97]:
# look at multiple templates for a specified channel
max_chans = temps.ptp(1).argmax(1)
max_chan = 10

idx = np.where(max_chans==max_chan)[0]
ctr=0
for id_ in idx:
    plt.plot((geom[:,0][:,None]+np.arange(101)/2.).T, 
         temps[id_] +
         geom[:,1][None]/3.,c=colors[ctr])
    ctr+=1
plt.text(geom[max_chan,0],geom[max_chan,1]/3.,str(max_chan),fontsize=10)
plt.show()

In [5]:
# grab all raw waveveforms for spikes on a single channel:
channel = 10

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'
filename = '/media/cat/4TBSSD/data/retina/2009/standardized.bin'
n_channels = 512
n_times = 101

idx2 = np.where(max_chans==channel)[0]
wfs_array = []
for id_ in idx2:
    
    idx_3 = np.where(spike_train[:,1]==id_)[0]
    spikes = spike_train[idx_3,0]
    idx4 = np.where(spikes<(300*20000-100))[0]
    spikes = spikes[idx4]-30
   
    wfs = binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32')
    print ("Neuron: ", id_, "waveform shape [n_spikes, n_times, n_chans]: ", wfs.shape)
    wfs_array.append(wfs)


Neuron:  1393 waveform shape [n_spikes, n_times, n_chans]:  (2315, 101, 512)
Neuron:  1502 waveform shape [n_spikes, n_times, n_chans]:  (1056, 101, 512)
Neuron:  1536 waveform shape [n_spikes, n_times, n_chans]:  (177, 101, 512)


In [138]:
# plot multiple spikes from mulitple neurons
units = [0,1,2]
for unit in units:
#     plt.plot((geom[:,0][:,None]+np.arange(101)/2.).T, 
#              temps[idx2[unit]] +
#              geom[:,1][None]/3.,linewidth=5, c=colors[unit])
    
    for k in range(10):
        plt.plot((geom[:,0][:,None]+np.arange(101)/2.).T, 
             wfs_array[unit][k] +
             geom[:,1][None]/3.,c=colors[unit],alpha=.4)
    
plt.text(geom[max_chan,0],geom[max_chan,1]/3.,str(max_chan),fontsize=10)
plt.show()




In [176]:
# VISUALIZE SORTED SPIKES using PCA
pca = PCA(n_components=2)

# compute PCS using all channels
print ("PCA ON ALL CHANNELS")
all_data = np.vstack(wfs_array)
print ("Stacked matrix of all spikes for all neurons on particualr channel: ", all_data.shape)
all_data1D = all_data.reshape(all_data.shape[0],-1)
print ("Flattened version for PCA: ", all_data1D.shape)

pcs_all_chans = pca.fit_transform(all_data1D) 
print ("post pca result: ", pcs_all_chans.shape)
print ("")

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
print ("PCA ON NEIGHBOUR CHANNELS")
pca = PCA(n_components=2)
locs = geom[channel]
chans_neighbour = []
thresh = 70 # distance in microns for channels to consider
for k in range(geom.shape[0]):
    dist = np.linalg.norm(locs-geom[k])
    if dist < thresh:
        chans_neighbour.append(k)
print ("max chan: ", channel, "has chans_neighbour: ", chans_neighbour)

all_data1D_neighbour_chans = all_data[:,:,chans_neighbour].reshape(all_data.shape[0],-1)
print ("Flattened matrix of all spikes using neighbour chans only:", all_data1D_neighbour_chans.shape)

pca_neighbour_chans = pca.fit_transform(all_data1D_neighbour_chans) 
print ("post pca result", pca_neighbour_chans.shape)
print ("")

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
pca = PCA(n_components=2)
print ("PCA ON MAX CHANNEL")
all_data1D_max_chan = all_data[:,:,channel].reshape(all_data.shape[0],-1)
print ("Flattened matrix of all spikes using max chan only:", all_data1D_max_chan.shape)

pca_max_chan = pca.fit_transform(all_data1D_max_chan) 
#pca_max_chan = pca_local.fit_transform(all_data1D_max_chan)
print ("post pca result", pca_max_chan.shape)
      



PCA ON ALL CHANNELS
Stacked matrix of all spikes for all neurons on particualr channel:  (3548, 101, 512)
Flattened version for PCA:  (3548, 51712)
post pca result:  (3548, 2)

PCA ON NEIGHBOUR CHANNELS
max chan:  10 has chans_neighbour:  [2, 5, 6, 10, 13, 14, 18]
Flattened matrix of all spikes using neighbour chans only: (3548, 707)
post pca result (3548, 2)

PCA ON MAX CHANNEL
Flattened matrix of all spikes using max chan only: (3548, 101)
post pca result (3548, 2)


In [178]:
# PLOT PCA STUFF
fig=plt.figure()
plt.suptitle("PCA PLOTS ON CLUSTERED DATA")
ax = plt.subplot(2,2,1)
plt.title("PCA on all channels")
ctr=0
for k in range(len(wfs_array)):
    x = pcs_all_chans[ctr:ctr+wfs_array[k].shape[0],0]
    y = pcs_all_chans[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]
    
ax = plt.subplot(2,2,2)
plt.title("PCA on neighbour channels")
ctr=0
for k in range(len(wfs_array)):
    x = pca_neighbour_chans[ctr:ctr+wfs_array[k].shape[0],0]
    y = pca_neighbour_chans[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]

ax = plt.subplot(2,2,3)
plt.title("PCA on max channel")
ctr=0
for k in range(len(wfs_array)):
    x = pca_max_chan[ctr:ctr+wfs_array[k].shape[0],0]
    y = pca_max_chan[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]

           

In [162]:
# visualize data using UMAP
import umap

#for 
#embedding = umap.UMAP().fit_transform(digits.data)

embedding_max_chan = umap.UMAP().fit_transform(all_data1D_max_chan) 
print (embedding_max_chan.shape)

embedding_all_chan = umap.UMAP().fit_transform(all_data1D) 
print (embedding_all_chan.shape)

embedding_neighbour_chan = umap.UMAP().fit_transform(all_data1D_neighbour_chans) 
print (embedding_neighbour_chan.shape)



(3548, 2)
(3548, 2)
(3548, 2)


In [164]:
# PLOT UMAP PLOTS on CLUSTERED DATA
fig=plt.figure()
plt.suptitle("UMAP PLOTS on CLUSTERED DATA")

ax = plt.subplot(2,2,1)
plt.title("UMAP on all channels")
ctr=0
for k in range(len(wfs_array)):
    x = embedding_all_chan[ctr:ctr+wfs_array[k].shape[0],0]
    y = embedding_all_chan[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]
    
ax = plt.subplot(2,2,2)
plt.title("UMAP on neighbour channels")
ctr=0
for k in range(len(wfs_array)):
    x = embedding_neighbour_chan[ctr:ctr+wfs_array[k].shape[0],0]
    y = embedding_neighbour_chan[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]

ax = plt.subplot(2,2,3)
plt.title("UMAP on max channel")
ctr=0
for k in range(len(wfs_array)):
    x = embedding_max_chan[ctr:ctr+wfs_array[k].shape[0],0]
    y = embedding_max_chan[ctr:ctr+wfs_array[k].shape[0],1]
    plt.scatter(x,y,c=colors[k])
    ctr+=wfs_array[k].shape[0]

#3 SWITCH OVER THE DETECTED SPIKES NOW



In [3]:
# RETINAL DATASETS
# do the same thing on all raw spikes detected by NN approach
spike_train_detected = np.load('/media/cat/4TBSSD/data/retina/2009/spike_detections.npy')
print (spike_train_detected.shape)

channel=251
idx = np.where(spike_train_detected==channel)[0]
print ("There were ", idx.shape[0], " detected spikes")
spikes_detected_times = spike_train_detected[idx,0]

print ("Let's see what the missed spikes look like")



(10762737, 2)
There were  19769  detected spikes
Let's see what the missed spikes look like


In [11]:
# NEUROPIXEL DRIFT DATASETS
# post spike sorting
#temps = np.load('/media/cat/4TBSSD/data/retina/2009/templates_0sec.npy')
#print ("templates [n_neurons, n_times, n_channels]: ", temps.shape)

#spike_train = np.load('/media/cat/4TBSSD/data/retina/2009/spike_train.npy')
#print ("spike_train [times, neuron_ids]: ", spike_train.shape)

if False:
    spike_train_detected = np.load('/media/cat/4TBSSD/data/synthetic/run30_monotonic_3chans/ground_truth/spike_train_ground_truth.npy')
    print (spike_train_detected.shape)    
    channel=252
    idx = np.where(spike_train_detected[:,1]==channel)[0]
    print ("There were ", idx.shape[0], " detected spikes on chan: ", channel)
    spikes_detected_times = spike_train_detected[idx,0]
else:
    temps_gt = np.load('/media/cat/4TBSSD/data/synthetic/run31_monotonic_1.5chans_100units_0/ground_truth/templates_ground_truth.npy')
    spike_train_gt = np.load('/media/cat/4TBSSD/data/synthetic/run31_monotonic_1.5chans_100units_0/ground_truth/spike_train_ground_truth.npy')
    print (spike_train_gt.shape)
   
print ("Let's see what the missed spikes look like")

# raw data
geom = np.loadtxt('/media/cat/4TBSSD/data/synthetic/run26_random_walk/geom_384chans.txt')
print ("geom: ", geom.shape)

n_channels = 384

(63041, 2)
There were  294  in unit:  0
Let's see what the missed spikes look like
geom:  (384, 2)


In [42]:
# grab all raw waveveforms for detected spikes

root_dir = '/media/cat/12TB/insync_cm3746/Data/patrick/'
filename = '/media/cat/4TBSSD/data/synthetic/run31_monotonic_1.5chans_100units_0/data_int16.bin'
#n_channels = n_chans
n_times = 101
data_type = 'int16'

# load a particular unit + all units on that channel:
unit = 5
max_chans=temps_gt.ptp(1).argmax(1)
channel = max_chans[unit]

channel = 181

units = np.where(max_chans==channel)[0]
print ("# units is ", units.shape, "  on channel: ", channel)

spikes = []
for unit in units:
    idx = np.where(spike_train_gt[:,1]==unit)[0]
    print ("There were ", idx.shape[0], " spikes, in unit: ", unit)
    spikes.append(spike_train_gt[idx,0])
    
spikes = np.hstack(spikes)
idx = np.argsort(spikes)
spikes=spikes[idx]
idx4 = np.where(np.logical_and(spikes>0, spikes<(300*20000-100)))[0] # clip so we don't go over 5 minutes
spikes = spikes[idx4]  # shift the waveform a bit



#spikes = spikes[::20]
print ("spikes: ", spikes.shape)

print ("...loading waveforms...")
wfs_detected = binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type=data_type)
print ("waveform shape [n_spikes, n_times, n_chans]: ", wfs_detected.shape)



# units is  (2,)   on channel:  181
There were  573  spikes, in unit:  88
There were  382  spikes, in unit:  93
spikes:  (600,)
...loading waveforms...
waveform shape [n_spikes, n_times, n_chans]:  (600, 101, 384)


In [43]:
# plt.plot(wfs_detected[::10,:,251].T, c='black',alpha=.1)
# plt.show()

In [44]:
# find max chan

pca = PCA(n_components=2)

# compute PCS using all channels
print ("PCA ON ALL CHANNELS")
all_data_detected = wfs_detected
#all_data_detected = wfs_
print ("Stacked matrix of all spikes for all neurons on particualr channel: ", all_data_detected.shape)
all_data1D = all_data_detected.reshape(all_data_detected.shape[0],-1)
print ("Flattened version for PCA: ", all_data1D.shape)

pcs_all_chans_detected = pca.fit_transform(all_data1D) 
print ("post pca result: ", pcs_all_chans_detected.shape)
print ("")

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
print ("PCA ON NEIGHBOUR CHANNELS")
pca = PCA(n_components=2)
locs = geom[channel]
chans_neighbour = []
thresh = 70 # distance in microns for channels to consider
for k in range(geom.shape[0]):
    dist = np.linalg.norm(locs-geom[k])
    if dist < thresh:
        chans_neighbour.append(k)
print ("max chan: ", channel, "has chans_neighbour: ", chans_neighbour)

all_data1D_neighbour_chans = all_data_detected[:,:,chans_neighbour].reshape(all_data_detected.shape[0],-1)
print ("Flattened matrix of all spikes using neighbour chans only:", all_data1D_neighbour_chans.shape)

pca_neighbour_chans = pca.fit_transform(all_data1D_neighbour_chans) 
print ("post pca result", pca_neighbour_chans.shape)
print ("")

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
pca = PCA(n_components=2)
print ("PCA ON MAX CHANNEL")
all_data1D_max_chan = all_data_detected[:,:,channel].reshape(all_data_detected.shape[0],-1)
print ("Flattened matrix of all spikes using max chan only:", all_data1D_max_chan.shape)

#all_data1D_max_chan = pca.fit_transform(all_data1D_max_chan) 
pca_max_chan = pca.fit_transform(all_data1D_max_chan)
print ("post pca result", pca_max_chan.shape)
      

PCA ON ALL CHANNELS
Stacked matrix of all spikes for all neurons on particualr channel:  (600, 101, 384)
Flattened version for PCA:  (600, 38784)
post pca result:  (600, 2)

PCA ON NEIGHBOUR CHANNELS
max chan:  181 has chans_neighbour:  [174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187]
Flattened matrix of all spikes using neighbour chans only: (600, 1414)
post pca result (600, 2)

PCA ON MAX CHANNEL
Flattened matrix of all spikes using max chan only: (600, 101)
post pca result (600, 2)


In [45]:
# PLOT PCA STUFF ON ALL DETETCED EVENTS
fig=plt.figure()
plt.suptitle("PCA PLOTS - ON ALL DETECTED EVENTS")
ax = plt.subplot(2,2,1)
plt.title("PCA on all channels")
ctr=0
#for k in range(len(wfs_array)):
x = pcs_all_chans_detected[:,0]
y = pcs_all_chans_detected[:,1]

plt.scatter(x,y,c=np.arange(y.shape[0]),cmap='viridis',alpha=.2)
    
ax = plt.subplot(2,2,2)
plt.title("PCA on neighbour channels")
x = pca_neighbour_chans[:, 0]
y = pca_neighbour_chans[:, 1]
plt.scatter(x,y,c=np.arange(y.shape[0]),cmap='viridis',alpha=.2)

ax = plt.subplot(2,2,3)
plt.title("PCA on max channel")
x = pca_max_chan[:,0]
y = pca_max_chan[:,1]
plt.scatter(x,y,c=np.arange(y.shape[0]),cmap='viridis',alpha=.2)
plt.show()

In [ ]:
# OPTIONAL Viauzlie PCs vs. time


In [7]:
# visualize data using UMAP on ALL DETECTED EVENTS
all_data_detected = wfs_detected
all_data1D = all_data_detected.reshape(all_data_detected.shape[0],-1)

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
locs = geom[channel]
chans_neighbour = []
thresh = 70 # distance in microns for channels to consider
for k in range(geom.shape[0]):
    dist = np.linalg.norm(locs-geom[k])
    if dist < thresh:
        chans_neighbour.append(k)

all_data1D_neighbour_chans = all_data_detected[:,:,chans_neighbour].reshape(all_data_detected.shape[0],-1)

# compute PCS on neighbour chans within 70 microns (Or can change this parameter)
all_data1D_max_chan = all_data_detected[:,:,channel].reshape(all_data_detected.shape[0],-1)

# computing embeddings for all chans x all xpikes
# embedding_all_chan = umap.UMAP().fit_transform(all_data1D) 
# print (embedding_all_chan.shape)
print ("DONE loading 1D arrays")

DONE loading 1D arrays


In [8]:
# compute embeddings for just some of the chans
embedding_neighbour_chan = umap.UMAP().fit_transform(all_data1D_neighbour_chans) 
print (embedding_neighbour_chan.shape)

embedding_max_chan = umap.UMAP().fit_transform(all_data1D_max_chan) 
print (embedding_max_chan.shape)



(19769, 2)
(19769, 2)


In [14]:
# PLOT UMAP PLOTS on CLUSTERED DATA
fig=plt.figure()
plt.suptitle("UMAP PLOTS on DETECTED EVENTS")

ax = plt.subplot(2,2,1)
plt.title("UMAP on all channels - OUT OF MEMORY ERROR")
ctr=0
# for k in range(len(wfs_array)):
#     x = embedding_all_chan[ctr:ctr+wfs_array[k].shape[0],0]
#     y = embedding_all_chan[ctr:ctr+wfs_array[k].shape[0],1]
#     plt.scatter(x,y,c=colors[k])
#     ctr+=wfs_array[k].shape[0]
    
ax = plt.subplot(2,2,2)
plt.title("UMAP on neighbour channels")
x = embedding_neighbour_chan[:,0]
y = embedding_neighbour_chan[:,1]
plt.scatter(x,y,c=colors[1],alpha=.1)

ax = plt.subplot(2,2,3)
plt.title("UMAP on max channel")
ctr=0
x = embedding_max_chan[:,0]
y = embedding_max_chan[:,1]
plt.scatter(x,y,c=colors[2],alpha=.1)
plt.show()